In [ ]:
# ============================================
# Cell 1: Install dependencies
# ============================================
!pip install torch transformers sentence-transformers faiss-cpu flask pyngrok pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.2 MB/s eta 0:00:00


In [ ]:
# ============================================
# Cell 2: Import libraries
# ============================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from threading import Thread
import faiss
import numpy as np
import yaml
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [ ]:
#cell 3 knowledge base

# Embedded Knowledge Base (20 Documents)
# ============================================

KNOWLEDGE_BASE = [
    {
        "id": 1,
        "title": "User Registration and Account Management",
        "content": """Registering on Shoplite is the first step for both buyers and sellers to participate in the marketplace. The process has been carefully designed to be simple for everyday users while maintaining safeguards against fraud and misuse. Buyers begin by submitting their name, email address, and a secure password, with the option to add a mobile phone number for SMS alerts. Immediately after signing up, Shoplite sends a verification email that must be confirmed within 24 hours. Without this confirmation, the account remains inactive and cannot be used for purchases. Once verified, buyers gain full access to browsing, searching, purchasing, and participating in loyalty programs.

Sellers, however, undergo a more rigorous onboarding process. In addition to providing personal details, sellers must submit a valid business license, a registered tax identification number, and bank details for payouts. This verification process typically takes between two and three business days. During the review period, sellers can explore the dashboard but cannot upload products or receive orders. Once approved, sellers can configure their storefront, upload images and descriptions, and begin managing orders.

Shoplite also offers a centralized account management dashboard. Users can update their contact information, shipping addresses, and notification preferences at any time. Security settings such as two-factor authentication and password resets are available to ensure accounts remain safe. Linked account options, such as logging in with Google or Apple ID, provide convenience for those who prefer single sign-on. Account deletion is supported but takes up to seven business days to process, giving the support team time to confirm pending transactions. These measures ensure that accounts remain secure while providing flexibility and ease of use for all participants on the platform."""
    },
    {
        "id": 2,
        "title": "Product Search and Filtering Features",
        "content": """One of the defining features of Shoplite is its sophisticated product search and filtering system, which has been engineered to simplify discovery for buyers while improving product visibility for sellers. The search bar on the homepage supports natural language queries, so users can search by product names, categories, or even descriptive terms such as “wireless headphones under $50.” In addition, buyers can search by brand names or directly look up seller storefronts. For app users, voice-enabled search adds another level of accessibility, while barcode scanning allows instant comparisons between physical retail products and Shoplite listings.

Filtering tools are a major component of the experience. Once a search has been performed, buyers can refine results using a wide set of filters: price ranges, product ratings, availability, shipping speed, and seller location. Multiple filters can be applied simultaneously, and results can then be sorted by relevance, lowest price, highest price, newest listing, or best-selling item. This layered approach allows buyers to narrow thousands of products down to just a handful that meet their needs.

Personalization is another unique aspect of the system. Buyers can save searches and receive notifications if products matching their criteria drop in price, come back in stock, or launch in new variants. These alerts arrive via email, push notification, or SMS depending on account preferences. The combination of intuitive searching, precise filtering, and personalization makes Shoplite’s search system one of its most important differentiators, offering convenience to buyers and greater exposure to sellers"""
    },
    {
        "id": 3,
        "title": "Shopping Cart and Checkout Process",
        "content": """The shopping cart and checkout process on Shoplite has been optimized to handle multi-seller transactions with minimal friction. Buyers can add items from multiple sellers into a single cart, simplifying the process of purchasing diverse products in one order. Items remain in the cart until manually removed and are synchronized across devices, so buyers can browse on a computer, add products, and later complete checkout on the mobile app without losing progress. The cart also allows buyers to adjust quantities, select product variants, or move items into a “Save for Later” list for future purchase.

Checkout is divided into three clear stages. First, buyers confirm or add a shipping address. Returning customers can select from saved addresses, while new customers may enter a fresh delivery location. Second, buyers choose a payment method from saved cards, PayPal accounts, wallets, or bank transfer options. Finally, a review screen displays all products, shipping fees, applicable taxes, and discounts from promotional codes. Buyers must confirm this summary before payment is processed, at which point the order becomes final.

Transparency is central to this process. All fees and delivery timelines are calculated automatically and shown before payment, leaving no room for hidden costs. Orders can be canceled before payment confirmation but once completed, cancelations depend on seller policies. By combining multi-seller functionality with an intuitive flow, Shoplite reduces cart abandonment rates while providing buyers with a straightforward and secure checkout experienc"""
    },
    {
        "id": 4,
        "title": "Payment Methods and Security",
        "content": """Shoplite provides a wide range of payment methods to cater to buyers across different regions. Supported methods include credit and debit cards such as Visa, MasterCard, and American Express, along with PayPal, Apple Pay, Google Pay, and region-specific digital wallets. Bank transfers are available in certain areas where card penetration is lower. Buyers may store payment methods securely for faster checkout in future orders, while sellers receive payments directly to their registered bank accounts.

Security is treated as a top priority. Shoplite adheres to PCI-DSS standards, ensuring compliance with global payment processing requirements. Transactions are encrypted end to end, and sensitive details such as card numbers are never stored directly. Instead, tokenization replaces card data with randomized tokens, eliminating the risk of data exposure. Additionally, Shoplite uses 3D Secure authentication for transactions that appear unusual or high-risk, prompting buyers to confirm via one-time passcodes.

Fraud prevention is enhanced with AI-driven monitoring systems that analyze transaction patterns. Suspicious activities trigger alerts and may require manual review by the risk management team. Refunds are always processed back to the original payment method, ensuring consistency and accountability. Processing times vary depending on the method, with card refunds usually taking three to five business days and bank transfers up to seven. By combining payment flexibility with stringent security measures, Shoplite ensures that buyers and sellers can transact with confidence"""
    },
    {
        "id": 5,
        "title": "Order Tracking and Delivery",
        "content": """Order fulfillment is an area where Shoplite has invested heavily to create a smooth and transparent experience for customers. Every confirmed purchase generates a unique tracking ID, which buyers can use to monitor the status of their orders in real time. The system displays order stages clearly: processing, shipped, in transit, out for delivery, and delivered. These updates are visible on both the web platform and the mobile app.

Buyers can select from different shipping options at checkout, depending on their urgency and budget. Standard shipping typically takes three to seven business days, express shipping reduces this to one to three business days, and same-day delivery is offered in select metropolitan areas. Sellers can also enable free shipping for orders above a specific amount to attract customers.

Shoplite’s tracking system integrates directly with major local and international courier partners. This ensures that updates are accurate and consistent across different regions. Notifications are automatically sent via email, SMS, and push notification at key milestones, such as when an order is dispatched or out for delivery. In cases of missed deliveries, buyers are given 48 hours to reschedule or request redelivery. Failed deliveries automatically trigger either a refund or replacement order depending on buyer preference.

By combining reliable delivery partners with an integrated tracking system, Shoplite ensures buyers always know the status of their purchases, reducing disputes and increasing satisfaction"""
    },
    {
        "id": 6,
        "title": "Return and Refund Policies",
        "content": """Shoplite has developed clear return and refund policies that aim to protect buyers while also ensuring sellers are not unfairly penalized. Buyers are eligible for returns within a standard 30-day window from the date of delivery. To initiate a return, customers must log into their account, locate the relevant order under purchase history, and submit a return request specifying the reason. Once approved, the system generates a return authorization number which must be included with the returned package.

Products must be returned unused, with all original packaging and accessories. Items that show clear signs of wear or tampering may be rejected for refunds. Certain product categories such as groceries, flowers, personalized goods, and digital downloads are strictly non-returnable. For electronics and high-value goods, buyers may be asked to provide photos or videos documenting the defect before approval.

Refunds are processed within 5–7 business days after Shoplite’s logistics or the seller confirms receipt and inspection of the item. Refunds are always issued to the original payment method to reduce fraud. For defective items, incorrect deliveries, or goods damaged in transit, Shoplite covers the return shipping cost. For buyer’s remorse returns (such as change of mind), return shipping costs may be deducted from the refund.

Exchanges are supported in cases where the buyer wishes to replace a defective or wrong item with a correct version. If a replacement is unavailable, Shoplite issues a full refund. All return requests and progress updates are tracked through the buyer dashboard, providing full visibility. This structured policy balances customer satisfaction with operational fairness, creating trust across the platform.."""
    },
    {
        "id": 7,
        "title": "Product Reviews and Ratings",
        "content": """Customer reviews and ratings are an integral part of the Shoplite ecosystem, helping buyers make informed choices while giving sellers valuable feedback. Only buyers who have actually purchased a product are eligible to leave reviews, ensuring authenticity. Reviews may include a star rating from one to five, written text, and optional photos or videos to showcase the product in real use. Each product’s overall score is a weighted average, with recent reviews given higher significance to reflect current quality.

To maintain credibility, Shoplite implements a review moderation system. Automated filters block offensive language, spam, and unrelated content. Additionally, suspicious review patterns such as multiple reviews from the same IP address are flagged for human verification. Fake or manipulated reviews may result in account suspension for sellers or buyers involved.

Sellers are encouraged to engage with reviews, especially negative ones, by responding politely and offering resolutions. This public interaction demonstrates accountability and improves seller reputation. Buyers benefit from verified purchase tags that distinguish genuine reviews from unverified commentary.

Reviews also feed into Shoplite’s search and ranking algorithms. Products with consistently high ratings appear higher in search results, driving more visibility and sales. Conversely, sellers with poor feedback risk losing credibility and may face account review. By giving voice to customers while maintaining strict moderation, Shoplite ensures reviews remain a trustworthy source of guidance, fostering transparency and accountability across the marketplace."""
    },
    {
        "id": 8,
        "title": "Seller Account Setup and Management",
        "content": """Becoming a Shoplite seller requires going through a structured setup and verification process. Sellers begin by visiting the *Shoplite Seller Portal, where they provide personal and business details. Required documentation includes a valid **business license, a **registered tax identification number, and active *bank account details to receive payouts. This verification typically takes two to three business days.

During the waiting period, sellers can explore a limited version of the dashboard but cannot upload products or receive orders. Once approved, sellers gain access to the full Seller Dashboard, where they can:

Upload and manage product listings with detailed descriptions, pricing, and images.
Track and fulfill customer orders.
Access analytics and insights to measure sales performance.
Customize their storefront to reflect branding.
Compliance with Shoplite’s marketplace guidelines is mandatory. Sellers are prohibited from listing counterfeit goods, restricted products, or items that violate local laws. Repeated violations can result in penalties ranging from temporary suspension to permanent account termination. Shoplite also provides sellers with access to training resources, best practice guides, and community forums where they can learn from experienced merchants.

Account management tools allow sellers to update their business information, banking details, and tax documentation at any time. Notifications alert them of upcoming policy changes or account issues requiring attention. By combining a careful onboarding process with continuous support and transparent rules, Shoplite ensures that its marketplace maintains high standards while offering sellers every opportunity to succeed"""
    },
    {
        "id": 9,
        "title": "Inventory Management for Sellers",
        "content": """Shoplite’s inventory management system is designed to prevent stockouts and overselling while giving sellers full visibility into product performance. Within the Seller Dashboard, merchants can create Stock Keeping Units (SKUs) for each product, enabling them to track variants such as size, color, or edition. Sellers may also generate barcodes to streamline packaging and fulfillment processes.

For large catalogs, Shoplite offers bulk upload functionality via CSV files, allowing sellers to import hundreds of items at once. Once uploaded, inventory data synchronizes across the marketplace instantly, ensuring customers always see accurate stock levels. When stock for a product reaches a pre-defined threshold, sellers receive low-stock alerts. They can then restock promptly or temporarily mark products as “restocking soon” to reassure buyers.

The system also provides detailed reporting tools. Sellers can track fast-moving versus slow-moving items, seasonal demand trends, and historical sales data. This helps merchants make informed restocking and pricing decisions. Integration with the analytics dashboard means sellers can link sales patterns with inventory fluctuations, identifying which promotions drove stock depletion and which products may require discounts to clear space.

Out-of-stock products may either be hidden from search or displayed with an expected restock date, depending on seller preference. By giving merchants flexibility and clear visibility, Shoplite reduces canceled orders due to unavailable items while optimizing operational efficiency. Strong inventory control contributes directly to higher buyer satisfaction and repeat purchases"""
    },
    {
        "id": 10,
        "title": "Commission and Fee Structure",
        "content": """Shoplite operates on a transparent commission and fee model that ensures sustainability while remaining competitive for sellers. Commissions are category-based, with rates varying to reflect industry norms. For instance, electronics carry an 8% commission, fashion items are set at 12%, and home goods typically fall at 10%. These rates are automatically applied at the point of sale, eliminating manual calculations for sellers.

In addition to commission, every transaction includes a flat $0.30 processing fee that covers payment handling and platform maintenance. Sellers are not charged listing fees, meaning they can upload unlimited products without upfront cost. Premium sellers, who subscribe to Shoplite’s advanced membership plan, enjoy reduced commission rates, greater promotional visibility, and access to exclusive campaigns such as seasonal sales.

Payouts are issued to sellers every 14 days, with commissions and fees already deducted. Detailed payout reports provide breakdowns of gross sales, deducted commissions, transaction fees, and the final net amount transferred. Sellers can view these reports directly in their dashboards or export them for bookkeeping. For sellers operating internationally, currency conversion fees may apply depending on the payout region.

By publishing all fees clearly and providing itemized reports, Shoplite builds trust with its sellers. The fee structure encourages sellers to scale their businesses while ensuring that the platform continues to invest in infrastructure, marketing, and security. This balance between fairness and sustainability is one of the reasons Shoplite attracts both small entrepreneurs and large retailers.

"""
    },
    {
        "id": 11,
        "title": "Customer Support Procedures",
        "content": """Shoplite’s customer support system is designed to provide quick and reliable assistance to both buyers and sellers, recognizing that a responsive help desk is critical for building trust in any online marketplace. Support is available 24/7 across multiple channels, including live chat, email, phone support, and a comprehensive self-service help center. Live chat is the most popular option for quick queries, allowing buyers to receive real-time assistance on matters such as delayed deliveries or refund status. Email support is intended for less urgent cases and typically has a response time of 12 to 24 hours. For high-priority situations such as payment disputes, sellers can access a dedicated hotline for faster resolution.

The platform uses a structured ticketing system to manage issues. Buyers may open tickets for problems like missing deliveries, defective products, or refund delays. Sellers, on the other hand, may request assistance for payout issues, account suspensions, or policy clarifications. Tickets are prioritized based on severity, and customers are notified of progress updates at each stage.

Shoplite’s escalation model ensures fairness. General support agents handle most cases at Tier 1, but if an issue involves compliance or repeated disputes, it is escalated to Tier 2 specialists. Legal and policy violations are forwarded to Tier 3 compliance officers. Documentation is often required, such as order confirmation numbers, proof of payment, or photos of damaged items. This ensures decisions are based on clear evidence.

By combining multiple support channels, a transparent ticketing system, and clear escalation paths, Shoplite ensures that customer concerns are addressed promptly and thoroughly. This commitment to responsive service strengthens the trust relationship between buyers, sellers, and the platform."""
    },
    {
        "id": 12,
        "title": "Mobile App Features",
        "content": """The Shoplite mobile application provides a comprehensive shopping experience tailored for customers and sellers who prefer managing their accounts and purchases on the go. While it mirrors the core functionality of the website, the app introduces mobile-first features that enhance convenience. Buyers can browse the entire catalog, add items to their cart, and complete purchases directly within the app. One standout feature is push notifications, which instantly alert users about delivery updates, restocks, and promotional events.

The app also supports innovative search methods. Buyers can scan product barcodes in physical stores to check availability and pricing on Shoplite. Voice search enables quick product discovery without typing, which is especially useful for mobile users. Security is reinforced with biometric login options such as fingerprint or facial recognition, making authentication fast and reliable.

Offline functionality allows users to continue browsing previously loaded product pages and saved carts, even without an internet connection. This ensures continuity in shopping sessions, with updates applied automatically once the device reconnects. Sellers benefit from mobile tools as well, including the ability to upload product listings using the phone camera, monitor real-time sales performance, and receive stock alerts directly on their device.

Mobile-exclusive campaigns also encourage downloads. Shoplite frequently releases app-only vouchers, discounts, or early access to sales events, rewarding buyers who prefer mobile engagement. For sellers, push alerts about low inventory or new orders allow for faster decision-making and better responsiveness to customer demand.

By combining user-friendly design, powerful search tools, and seller management features, the Shoplite app extends the platform’s reach into everyday mobility, ensuring that Shoplite remains accessible anytime and anywhere"""},
    {
        "id": 13,
        "title": "API Documentation for Developers",
        "content": """To enable integrations and third-party solutions, Shoplite provides developers with a robust RESTful API. These APIs allow external systems to connect seamlessly with Shoplite for managing inventory, processing orders, and retrieving analytics. Authentication requires an API key, which developers can request after registering through the developer portal. To ensure security and system stability, API requests are subject to rate limits, typically capped at 500 requests per minute.

The API supports a wide range of endpoints. Sellers can automate inventory updates, bulk upload products, or retrieve order details for integration into external ERP systems. Payment-related endpoints provide transaction histories and reconciliation data, which can be exported for accounting purposes. Analytics endpoints allow developers to pull sales trends, customer demographics, and conversion data. Responses are standardized in JSON format, with clear error codes and messages to guide troubleshooting.

Shoplite provides a dedicated sandbox environment for developers to test applications before moving them to production. This environment mirrors live functionality without processing financial transactions, ensuring developers can safely experiment and refine integrations.

Comprehensive developer documentation is available, including quick-start guides, code samples, and SDKs for popular programming languages. A community forum and dedicated support team provide additional assistance, while critical bugs or integration issues may be escalated to engineering staff.

By providing a well-documented, secure, and flexible API, Shoplite enables sellers, partners, and third-party developers to extend platform functionality, automate workflows, and innovate on top of the core marketplace infrastructure."""
    },
    {
        "id": 14,
        "title": "Security and Privacy Policies",
        "content": """Security and privacy are foundational principles at Shoplite, and the company has invested heavily in systems and processes that protect user data while complying with international standards. All data transmitted between users and the platform is secured using end-to-end encryption. Sensitive information, including passwords and payment details, is stored in encrypted form, and additional tokenization ensures that raw data is never exposed.

Shoplite complies with global data protection regulations such as the General Data Protection Regulation (GDPR) in Europe and the California Consumer Privacy Act (CCPA) in the United States. Users have full control over their privacy settings, including the ability to opt out of marketing emails, disable data tracking for analytics, and request data exports or permanent account deletion.

Regular third-party audits are conducted to test for vulnerabilities. Shoplite also runs a bug bounty program that rewards ethical hackers for identifying potential security flaws. Internally, security teams perform quarterly penetration tests and vulnerability scans to ensure evolving threats are addressed proactively.

Privacy principles dictate that Shoplite never sells customer data to third parties. Data may be used in aggregated and anonymized formats to generate insights, but personally identifiable information is never shared without user consent. Cookies are limited to essential and analytics functions, and users can adjust cookie preferences at any time.

By prioritizing compliance, encryption, transparency, and continuous monitoring, Shoplite provides users with confidence that their personal and financial data remain safe at all times"""
    },
    {
        "id": 15,
        "title": "Promotional Codes and Discounts",
        "content": """Promotional codes and discounts are central to Shoplite’s strategy for driving engagement and rewarding loyal buyers. These incentives come in several formats. Percentage-based discounts reduce the total order cost, while fixed-amount vouchers offer savings on minimum spends. Free shipping vouchers are also popular, particularly during seasonal campaigns. Each promotional code comes with an expiration date and defined conditions of use, such as minimum order value or category-specific restrictions.

Buyers can apply promotional codes during checkout or save them in their account wallet for future use. Codes may be single-use, applicable only once per buyer, or multi-use, allowing redemption multiple times during an active campaign. Shoplite also operates platform-wide promotions during major retail events like Black Friday and Cyber Monday, offering substantial discounts to attract new customers.

Sellers are able to generate their own promotional campaigns, tailoring discounts to specific products or customer groups. For example, a seller may run a “buy one, get one free” event or provide a discount for bulk purchases. Shoplite’s system ensures that seller-generated promotions cannot conflict with platform-wide campaigns, preserving consistency.

To prevent abuse, Shoplite employs fraud detection systems that identify unusual redemption patterns. Users who attempt to exploit codes through multiple accounts risk suspension. Despite these safeguards, the majority of promotional activities are viewed positively, with data showing increased conversion rates and repeat purchases.

By offering flexible and well-regulated promotional tools, Shoplite balances the needs of buyers looking for value with the interests of sellers aiming to boost visibility and sales.."""
    },
    {
        "id": 16,
        "title": "Seller Analytics and Insights",
        "content": """Shoplite equips sellers with detailed analytics to help them make informed business decisions. The analytics dashboard provides sellers with a comprehensive overview of their store performance, including sales figures, customer demographics, and traffic sources. Sellers can view revenue broken down daily, weekly, or monthly, enabling them to identify seasonal trends and make timely adjustments to their marketing strategies.

Key performance indicators include conversion rates, abandoned cart rates, and product-level sales breakdowns. Sellers can also see how their products perform compared to category averages, giving them valuable benchmarks. Customer insights highlight purchasing behavior, regional preferences, and loyalty patterns, which are useful for tailoring promotions or expanding product lines.

Traffic data reveals whether visits come from organic searches, paid ads, or repeat customers. This helps sellers assess the effectiveness of advertising campaigns and allocate budgets more efficiently. Export options in CSV and PDF formats allow sellers to integrate these insights into external business tools. Weekly automated summaries are sent by email, providing quick snapshots without logging in.

By providing actionable data, Shoplite empowers sellers to optimize pricing, manage inventory strategically, and enhance customer engagement. Over time, these insights translate into better customer retention and higher profitability."""
    },
    {
        "id": 17,
        "title": "Shipping Policies and Partners",
        "content": """Shipping reliability is central to customer satisfaction, and Shoplite has established clear shipping policies supported by strong partnerships with courier services. Buyers can select from standard, expedited, or same-day delivery depending on location and urgency. Standard shipping typically takes three to seven business days, while expedited shipping reduces this to one to three days. Same-day delivery is available in select metropolitan regions, where partnered couriers have the capacity to fulfill orders quickly.

Shipping costs are calculated automatically at checkout based on product weight, dimensions, and destination. Sellers may choose to subsidize or eliminate shipping costs for orders above certain thresholds, a strategy that often drives higher basket sizes. Free shipping campaigns are also run during seasonal events to increase customer loyalty.

All packages are insured against loss or damage. If a shipment is lost in transit or arrives damaged, buyers are guaranteed a replacement or full refund. Tracking IDs are issued for every order, and buyers receive regular updates via email, SMS, and app notifications.

Shoplite partners with a mix of local couriers for domestic deliveries and international logistics providers for cross-border orders. This hybrid approach allows the platform to scale globally while ensuring local efficiency. Through clear policies and reliable partnerships, Shoplite minimizes delivery risks and ensures buyers receive their orders on time.."""
    },
    {
        "id": 18,
        "title": "Fraud Prevention and Security Features",
        "content": """Fraud prevention is one of the cornerstones of Shoplite’s operations, and significant investment has been made into ensuring the marketplace remains a safe environment for both buyers and sellers. Online marketplaces often face risks such as payment fraud, counterfeit product listings, and account takeovers. To combat this, Shoplite employs a multi-layered defense system that blends technology, strict policies, and human oversight.

For buyers, the most important safeguard is the Purchase Guarantee. If a product is not delivered, is misrepresented, or is discovered to be counterfeit, buyers are eligible for a full refund. Payments are closely monitored with AI-driven fraud detection systems that flag suspicious transactions in real time. Examples include unusually high-value purchases, bulk orders of the same product from new accounts, or inconsistent delivery addresses. In such cases, buyers may be asked to verify their identity through one-time passcodes or additional authentication steps.

Sellers are also subject to rigorous checks. Before payouts are released, sellers must provide valid government-issued identification and banking details, which are cross-verified. Shoplite’s system monitors accounts for patterns such as repeated chargebacks, unusually high refund requests, or abnormal spikes in sales activity. Accounts that raise red flags are escalated for manual review by Shoplite’s risk management team.

Education is another preventive measure. Shoplite regularly publishes best-practice guides for sellers to recognize fraudulent buyers and encourages buyers to shop only from verified sellers. Regular updates to fraud models ensure Shoplite evolves with new threats. In cases where fraud is confirmed, Shoplite acts decisively by suspending accounts, withholding payouts, and collaborating with law enforcement if necessary. By combining automated monitoring, human review, and clear refund guarantees, Shoplite maintains integrity and ensures that genuine users can transact with confidence"""
    },
    {
        "id": 19,
        "title": "Seller Payouts and Withdrawals",
        "content": """Shoplite understands that sellers, whether small businesses or large retailers, depend on timely and predictable payouts to maintain healthy cash flow. To support this, the platform operates on a transparent withdrawal system with regular payout schedules, clear reporting tools, and robust error handling. Standard payouts are issued every 14 days, while premium sellers subscribed to Shoplite’s advanced plan may opt for weekly payments. Funds are deposited directly into the bank account specified during seller registration, provided the account passes verification checks.

A minimum payout threshold of $50 applies. If sellers do not reach this amount in a payout cycle, earnings roll over to the next period. Each payout includes a detailed report accessible via the Seller Dashboard. These reports break down gross sales, commissions, transaction fees, refunds, and the final net amount transferred. Reports can be exported in CSV or PDF formats, making it easy for sellers to reconcile their Shoplite transactions with their accounting systems.

For international sellers, payouts are supported in multiple currencies, with automatic conversion applied where necessary. Currency conversion fees are displayed transparently to avoid confusion. Failed payouts, often caused by outdated or incorrect bank details, are automatically retried within three business days. During this time, sellers receive email notifications prompting them to update their banking information.

Refunds and chargebacks are handled fairly by deducting the owed amounts from upcoming payouts, ensuring buyers are protected without placing an immediate financial burden on sellers. Sellers can view all chargeback details within their reports, allowing them to dispute claims if necessary.

By providing reliable payment cycles, transparent reporting, and clear communication, Shoplite ensures sellers can focus on growing their businesses rather than worrying about when or how they will be paid. This trust is central to maintaining a thriving seller community"""
    },
    {
        "id": 20,
        "title": "Loyalty Program and Buyer Rewards",
        "content": """Shoplite’s loyalty program has been developed to reward consistent buyers and transform occasional shoppers into long-term customers. The program is based on a points system, where buyers earn points for every dollar spent on the platform. These points accumulate in their account and can be redeemed for vouchers, discounts, or shipping benefits. During promotional campaigns, such as holiday sales, Shoplite awards bonus points, allowing members to climb tiers faster and access greater benefits.

The program is structured around membership tiers. Standard members earn points at the base rate, while higher tiers—Silver, Gold, and Platinum—unlock progressively more attractive rewards. Silver members may receive small birthday discounts, Gold members often gain free express shipping on selected products, and Platinum members enjoy exclusive early access to new product launches and invitation-only sales events. This tiered system not only encourages spending but also fosters a sense of exclusivity and recognition among loyal customers.

Redeeming points is straightforward. During checkout, buyers can choose whether to apply points to reduce their order total or save them for bigger rewards later. Points are valid for 12 months from the date of issue, ensuring that customers are encouraged to use them rather than let them accumulate indefinitely. If points remain unused past expiration, they are automatically forfeited.

Fraud prevention measures ensure the program remains fair. Shoplite tracks suspicious behavior, such as creating multiple accounts to exploit point-based promotions. Accounts flagged for abuse risk losing loyalty benefits entirely. Shoplite also uses AI monitoring to detect unnatural redemption patterns, such as excessive voucher conversions.

Ultimately, the loyalty program functions as both a marketing tool and a customer retention strategy. By rewarding regular engagement and offering exclusive perks, Shoplite ensures buyers feel valued and motivated to continue shopping on the platform, reinforcing the bond between the marketplace and its community."""
    }
]

In [ ]:
# ============================================
# Cell 4: Structured Prompts
# ============================================
PROMPTS_YAML = """
version: "1.0"
created: "2025-09-23"
author: "Your Name"

# --- Core prompts ---

base_retrieval_prompt:
  role: |
    You are a helpful Shoplite customer service assistant.
  goal: |
    Answer accurately using ONLY the provided Shoplite documents.
  context_guidelines:
    - Do not invent facts. If the answer is not in context, say you don’t have it.
    - Cite the exact document titles you used.
    - Keep the final answer to ≤2 sentences, plain text (no lists or headings).
  response_format: |
    Answer: [final answer only—no preface]
    Sources: [comma-separated document titles]

multi_document_prompt:
  role: |
    You are a Shoplite assistant specializing in multi-document synthesis.
  goal: |
    Combine relevant facts from multiple Shoplite documents into a single coherent answer.
  context_guidelines:
    - Reconcile overlaps consistently; prefer the most specific policy statement.
    - Keep the final answer to ≤3 sentences, plain text (no lists/headings).
    - Cite all documents actually used.
  response_format: |
    Answer: [synthesized answer across docs]
    Sources: [all document titles used]

no_context_prompt:
  role: |
    You are a cautious Shoplite assistant that refuses unsupported answers.
  goal: |
    If the context doesn’t contain the answer, say so and suggest what to ask instead.
  context_guidelines:
    - No speculation or external knowledge.
  response_format: |
    Answer: Sorry, I couldn’t find that in the provided Shoplite documentation. Please ask about Shoplite features, policies, or services.
    Sources: None

clarification_prompt:
  role: |
    You ask for just enough clarification to locate the right documents.
  goal: |
    Request 1–2 targeted clarifications tied to likely documents.
  context_guidelines:
    - Keep it short; no assumptions yet.
  response_format: |
    Answer: Your question could refer to [payments (Document 4: Payment Methods and Security)] or [fraud checks (Document 18: Fraud Prevention and Security)]. Which one do you mean?
    Sources: None


strict_format_prompt:
  role: |
    You enforce output format.
  goal: |
    Ensure the final output exactly matches:
    - "Answer: <one or two sentences, plain text>"
    - "Sources: <document titles only>"
  context_guidelines:
    - No prefaces like "Explanation" or "Reasoning".
    - No bullet points, markdown, or headings.
  response_format: |
    Answer: [1–2 sentences, plain text only]
    Sources: [document titles only]

numeric_truth_prompt:
  role: |
    You are precise about numbers and time windows.
  goal: |
    Quote only numbers and durations that appear in the context; otherwise state unknown.
  context_guidelines:
    - Prefer exact figures (e.g., "30 days", "5–7 business days").
    - If a number is missing, say "not specified in the provided documentation."
  response_format: |
    Answer: [≤2 sentences with exact numbers if present; otherwise say not specified]
    Sources: [documents used]

policy_plus_procedure_prompt:
  role: |
    You explain both the rule and the steps a user must take.
  goal: |
    Provide the policy in 1 sentence, then the procedure in 1 sentence.
  context_guidelines:
    - Use only policy/procedure from the documents (e.g., returns + tracking).
  response_format: |
    Answer: [sentence 1 = the rule] [sentence 2 = the steps]
    Sources: [documents used]

developer_api_prompt:
  role: |
    You answer developer questions using the API documentation context.
  goal: |
    Provide endpoint behavior, authentication, rate limits, and environment (sandbox vs production).
  context_guidelines:
    - Include rate limits and auth requirements if present.
  response_format: |
    Answer: [≤3 sentences focusing on endpoints, auth, limits, environments]
    Sources: [Document 13: API Documentation for Developers]

loyalty_tiers_prompt:
  role: |
    You explain Shoplite loyalty tiers crisply.
  goal: |
    Summarize tier benefits and timing in ≤2 sentences.
  context_guidelines:
    - Mention point validity/expiration if present.
  response_format: |
    Answer: [≤2 sentences summarizing tiers/benefits/timing]
    Sources: [Document 20: Loyalty Program and Buyer Rewards]

safety_guard_prompt:
  role: |
    You only answer Shoplite platform questions within scope.
  goal: |
    If the question is unrelated to Shoplite docs, refuse and redirect.
  context_guidelines:
    - Respect scope: Shoplite features, policies, seller tools, etc.
  response_format: |
    Answer: I can help with Shoplite features and policies covered in the documentation. Please ask about those topics.
    Sources: None
"""
PROMPTS = yaml.safe_load(PROMPTS_YAML)

In [ ]:
# ============================================
# Cell 5: Embeddings + FAISS Index
# ============================================
embedder = SentenceTransformer('all-MiniLM-L6-v2')
docs_embeddings = embedder.encode([doc["content"] for doc in KNOWLEDGE_BASE], convert_to_numpy=True).astype('float32')
# L2-normalize (makes distance comparisons more stable)
docs_embeddings /= (np.linalg.norm(docs_embeddings, axis=1, keepdims=True) + 1e-12)

index = faiss.IndexFlatIP(docs_embeddings.shape[1])   # cosine via dot after normalization
index.add(docs_embeddings)

def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True).astype('float32')
    q /= (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)
    D, I = index.search(q, k)
    return [KNOWLEDGE_BASE[i] for i in I[0]]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# ============================================
# Cell 6: Load LLM (Falcon-7B-Instruct with GPU)
# ============================================
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "tiiuae/falcon-7b-instruct"
print(f"Loading model {model_id}...")

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# ensure pad token exists to avoid warnings / truncation issues
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# choose dtype based on your hardware; float16 is broadly safe
dtype_choice = "float16"  # switch to "bfloat16" only if your GPU supports it well

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=getattr(torch, dtype_choice),
    low_cpu_mem_usage=True
)

model.config.use_cache = False
model.generation_config.use_cache = False

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=False,   # ← was True
    temperature=None,  # ← ignore when do_sample=False
    top_p=None,        # ← ignore when do_sample=False
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
def _gpu_summary():
    if torch.cuda.is_available():
        p = torch.cuda.get_device_properties(0)
        return f"cuda | {torch.cuda.get_device_name(0)} | VRAM {p.total_memory/1e9:.1f} GB"
    return "cpu"

print(f"[Model Ready] {model_id} on {_gpu_summary()} | dtype={next(model.parameters()).dtype} | pad_id={tokenizer.pad_token_id} | eos_id={tokenizer.eos_token_id}")

Loading model tiiuae/falcon-7b-instruct...


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Device set to use cuda:0


[Model Ready] tiiuae/falcon-7b-instruct on cuda | Tesla T4 | VRAM 15.8 GB | dtype=torch.float16 | pad_id=11 | eos_id=11


In [ ]:
def generate_answer(prompt: str, max_new_tokens: int = 160) -> str:
    """
    Generate answer with improved parsing to extract only the actual answer.
    """
    out = llm_pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=None,
        top_p=None,
        use_cache=False
    )[0]["generated_text"]

    # Get only the continuation after the prompt
    continuation = out[len(prompt):].strip()

    # ===== IMPROVED PARSING =====
    # Look for "Answer:" marker
    if "Answer:" in continuation:
        # Extract text after "Answer:" but before "Sources:"
        start = continuation.find("Answer:") + 7  # 7 = len("Answer:")
        end = continuation.find("Sources:")

        if end != -1:
            answer_text = continuation[start:end].strip()
        else:
            answer_text = continuation[start:].strip()

        # Clean up the extracted answer
        # Remove "Explanation:" and everything after it
        if "Explanation:" in answer_text:
            answer_text = answer_text.split("Explanation:")[0].strip()

        # Remove bullet points and meta-commentary
        lines = answer_text.split('\n')
        clean_lines = []

        for line in lines:
            line_stripped = line.strip()

            # Skip empty lines
            if not line_stripped:
                continue

            # Skip lines that are meta-commentary
            line_lower = line_stripped.lower()
            if any(phrase in line_lower for phrase in [
                "provided documents",
                "based on the information",
                "the answer should",
                "final answer should",
                "documents are the",
                "including the document"
            ]):
                continue

            # Skip bullet points starting with -
            if line_stripped.startswith('-'):
                continue

            # Keep substantial content
            clean_lines.append(line_stripped)

        if clean_lines:
            return ' '.join(clean_lines)

    # Fallback: return the raw continuation
    return continuation

In [ ]:
#cell 7: RAG PIPELINE

TOP_K_DOCS = 2
MAX_CONTEXT_CHARS = 300
MAX_TOKENS = 120

def rag_answer(query, prompt_type="base_retrieval_prompt"):
    retrieved_docs = retrieve(query, k=TOP_K_DOCS)

    if not retrieved_docs and prompt_type == "no_context_prompt":
        return PROMPTS["no_context_prompt"]["response_format"], []

    # Truncate each doc
    context = "\n".join([
        f"{doc['title']}: {doc['content'][:MAX_CONTEXT_CHARS]}..."
        for doc in retrieved_docs
    ])

    # ===== MORE EXPLICIT PROMPT =====
    structured_prompt = f"""Context Documents:
{context}

Question: {query}

CRITICAL INSTRUCTIONS:
1. Answer the question using ONLY information from the context above
2. Your response must be 1-2 complete sentences
3. Do NOT include any meta-commentary, explanations, or reasoning
4. Do NOT start with "Explanation:" or list the documents
5. Write the answer as if speaking directly to the user
6. If the answer is not in the context, say: "I don't have that information in the provided documentation."

Answer:""".strip()

    # Generate with more tokens to account for verbose models
    raw = generate_answer(structured_prompt, max_new_tokens=MAX_TOKENS)

    # The generate_answer function now handles cleaning
    return raw, retrieved_docs

In [ ]:
# ============================================
# Cell 8: Flask API
# ============================================
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json or {}
    query = data.get("query", "").strip()
    if not query:
        return jsonify({"error": "query is empty"}), 400
    prompt_type = data.get("prompt_type", "base_retrieval_prompt")
    answer, docs = rag_answer(query, prompt_type=prompt_type)
    return jsonify({
        "query": query,
        "answer": answer,
        "sources": [doc["title"] for doc in docs]
    })

@app.route('/ping', methods=['GET'])
def ping():
    return jsonify({"status": "LLM is running"})

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "documents_loaded": len(KNOWLEDGE_BASE)})
def run_flask():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)


In [ ]:
# ============================================
# Cell 9: ngrok input

# Enter ngrok token
ngrok_token = input("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)

# Start ngrok tunnel
public_url = ngrok.connect(5000)
public_url_str = public_url.public_url
print("Public URL:", public_url_str)


thread = Thread(target=run_flask,daemon=True)
thread.start()


Enter your ngrok token: 33LMxzmViaImJI0HLyXWft8Ceac_XQ77zxeW2f1mZwH9Qgvn
Public URL: https://unpercussed-ean-uncircularized.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
# ============================================
#cell 10 :Test API endpoints

import requests

url = f"{public_url_str}/chat"
query = {
    "query": "What is the return window for Shoplite products?",
    "prompt_type": "base_retrieval_prompt"
}

# Increase timeout to 2 minutes
resp = requests.post(url, json=query, timeout=120)
print(resp.json())






INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 11:55:01] "POST /chat HTTP/1.1" 200 -


{'answer': 'The return window for Shoplite products is 30 days from the date of delivery.', 'query': 'What is the return window for Shoplite products?', 'sources': ['Return and Refund Policies', 'Seller Account Setup and Management']}
